In [2]:
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import pandas as pd
import numpy as np
import warnings
import pickle
import os

pd.set_option("display.max_columns", 100)
warnings.filterwarnings("ignore")

# EDA

The dataset used is the result of the merging between the Telco Datasets by IBM on Kaggle [IBM Telco Churn Dataset](https://www.kaggle.com/datasets/aadityabansalcodes/telecommunications-industry-customer-churn-dataset/data)

In [3]:
df = pd.read_csv('data/Telecom_Customer_Churn.csv', header=0, index_col=0)
df.head()

,Customer ID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason,LoyaltyID,Tenure,Churn,Age,Under 30,Married,Number of Dependents,Location ID,ID,Population,Service ID,Quarter,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Avg Monthly Long Distance Charges,Internet Type,Avg Monthly GB Download,Device Protection Plan,Premium Tech Support,Streaming Music,Unlimited Data,Monthly Charge,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Status ID,Satisfaction Score,Customer Status,Churn Category
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer,326527,2,Yes,37,No,No,0,PZZUIT6837,3,"58,198",ZLZTZWPWE3351,Q3,No,0,2,NaN,10.47,DSL,21,No,No,No,Yes,53.85,0.0,0,20.94,129.09,SUDNGT6444,1,Churned,Competitor
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved,503388,2,Yes,19,Yes,No,2,CKNALR2542,5,"43,019",GXJVSQIYH4715,Q3,No,0,2,NaN,9.12,Fiber Optic,51,No,No,No,Yes,70.70,0.0,0,18.24,169.89,KZSZDV8891,2,Churned,Other
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved,160192,8,Yes,31,No,No,2,VCQKYE1638,6,"62,784",KDUHHRQBI5663,Q3,No,0,8,NaN,12.15,Cable,26,Yes,No,Yes,Yes,99.65,0.0,0,97.20,917.70,EPTIUU1269,3,Churned,Other
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved,582674,28,Yes,23,Yes,Yes,3,SNLISG8671,9,"1,957",WXXQTNEZW3786,Q3,No,0,28,Offer C,4.89,Fiber Optic,47,Yes,Yes,Yes,Yes,104.80,0.0,0,136.92,3182.97,PAJIVH8196,3,Churned,Other
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices,933642,49,Yes,38,No,No,1,YTSGZA8582,14,"15,140",LTNFDWAMC2575,Q3,No,0,49,NaN,44.33,Fiber Optic,11,Yes,No,Yes,Yes,103.70,0.0,0,2172.17,7208.47,RXFOMV1173,1,Churned,Competitor


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7042
Data columns (total 65 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Count                              7043 non-null   int64  
 2   Country                            7043 non-null   object 
 3   State                              7043 non-null   object 
 4   City                               7043 non-null   object 
 5   Zip Code                           7043 non-null   int64  
 6   Lat Long                           7043 non-null   object 
 7   Latitude                           7043 non-null   float64
 8   Longitude                          7043 non-null   float64
 9   Gender                             7043 non-null   object 
 10  Senior Citizen                     7043 non-null   object 
 11  Partner                            7043 non-null   object 
 1

## Data Processing

From the .info() output we can state the resulting dataset is consistent (7043 values in total) and, nearly, without NULL values.
Only few columns present some NULL values:
- *Offer*
- *Churn Reason*
- *Internet Type*
- *Churn Category*

For this columns must be evaluated ow to deal with the missing values and how to impute them, if necessary

For the column *Offer* can be set the missing values as **None** ince the customer can or cannot have used an offer

In [5]:
df['Offer'].fillna('None', inplace=True)

Internet Type column is very similar to Internet Service column, so it can be used to fill the missing values or can be considered to be droped

In [6]:
df[['Internet Service', 'Internet Type']]

,Internet Service,Internet Type
0,DSL,DSL
1,Fiber optic,Fiber Optic
2,Fiber optic,Cable
3,Fiber optic,Fiber Optic
4,Fiber optic,Fiber Optic
...,...,...
7038,No,NaN
7039,DSL,Cable
7040,Fiber optic,Fiber Optic
7041,DSL,DSL


From the code Below can be observed that when *Churn Category* is **NULL** also *Churn Reason* is **NULL**. 
By combining this with the values in the column *Churn* cn be seen that this happen whenever the customer doeasn't churn from the company, so the missing values can be resolved by replacing the missing value with a new placeholder like "Note Churned".

In [7]:
df[['Churn','Churn Category', 'Churn Reason']].loc[(df['Churn Category'].isnull())]

,Churn,Churn Category,Churn Reason
1869,No,NaN,NaN
1870,No,NaN,NaN
1871,No,NaN,NaN
1872,No,NaN,NaN
1873,No,NaN,NaN
...,...,...,...
7038,No,NaN,NaN
7039,No,NaN,NaN
7040,No,NaN,NaN
7041,No,NaN,NaN


In [8]:
df['Churn Reason']

0        Competitor made better offer
1                               Moved
2                               Moved
3                               Moved
4       Competitor had better devices
                    ...              
7038                              NaN
7039                              NaN
7040                              NaN
7041                              NaN
7042                              NaN
Name: Churn Reason, Length: 7043, dtype: object

In [10]:
df['Churn Category'].value_counts()

Churn Category
Competitor         841
Attitude           314
Dissatisfaction    303
Price              211
Other              200
Name: count, dtype: int64

In [11]:
df.head()

,Customer ID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason,LoyaltyID,Tenure,Churn,Age,Under 30,Married,Number of Dependents,Location ID,ID,Population,Service ID,Quarter,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Avg Monthly Long Distance Charges,Internet Type,Avg Monthly GB Download,Device Protection Plan,Premium Tech Support,Streaming Music,Unlimited Data,Monthly Charge,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Status ID,Satisfaction Score,Customer Status,Churn Category
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer,326527,2,Yes,37,No,No,0,PZZUIT6837,3,"58,198",ZLZTZWPWE3351,Q3,No,0,2,None,10.47,DSL,21,No,No,No,Yes,53.85,0.0,0,20.94,129.09,SUDNGT6444,1,Churned,Competitor
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved,503388,2,Yes,19,Yes,No,2,CKNALR2542,5,"43,019",GXJVSQIYH4715,Q3,No,0,2,None,9.12,Fiber Optic,51,No,No,No,Yes,70.70,0.0,0,18.24,169.89,KZSZDV8891,2,Churned,Other
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved,160192,8,Yes,31,No,No,2,VCQKYE1638,6,"62,784",KDUHHRQBI5663,Q3,No,0,8,None,12.15,Cable,26,Yes,No,Yes,Yes,99.65,0.0,0,97.20,917.70,EPTIUU1269,3,Churned,Other
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved,582674,28,Yes,23,Yes,Yes,3,SNLISG8671,9,"1,957",WXXQTNEZW3786,Q3,No,0,28,Offer C,4.89,Fiber Optic,47,Yes,Yes,Yes,Yes,104.80,0.0,0,136.92,3182.97,PAJIVH8196,3,Churned,Other
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices,933642,49,Yes,38,No,No,1,YTSGZA8582,14,"15,140",LTNFDWAMC2575,Q3,No,0,49,None,44.33,Fiber Optic,11,Yes,No,Yes,Yes,103.70,0.0,0,2172.17,7208.47,RXFOMV1173,1,Churned,Competitor
